In [3]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 10
np.set_printoptions(precision=4, suppress=True)
import json

import glob, os
import re

import subprocess

In [4]:
pathfolder = ''
#example path:
#C:\\Users\\MSI\\Desktop\\Thesis\\NewDataSets\\rumoureval2019\\rumoureval-2019-test-data
#The input here is the path forRumorEval dataset, more specifically the folder in it called "rumoureval-2019-test-data",

annotation = pd.read_json('')
#example path:
#C:\\Users\\MSI\\Desktop\\Thesis\\NewDataSets\\rumoureval2019\\final-eval-key.json
#The paths may be different, but the destination of the path needs to be "rumoureval2019\\final-eval-key.json"

export_destination = ''
#example path:
#'C:\\Users\\MSI\\Desktop\\Thesis\\Results\\test.tsv'
#Note: This can be anywhere, but the name must be "test.tsv"

trib_results = ''
#example path:
#'C:\\Users\\MSI\\Desktop\\Thesis\\Results\\phoeme\\Trib_results.csv'
#Note: This value takes the result of Tribrid_pos, so make sure that the name set on Tribrid_pos is the same as here

In [5]:
replyfoldername = "replies"

sourcefoldername = "source-tweet"

structurejsonname = "structure.json"

In [6]:
pos_support_label = 'support'
neg_support_label = 'deny'

In [7]:
display(annotation)
if annotation.loc['7g16as']['subtaskaenglish'] == pos_support_label: print("yeah")

,subtaskaenglish,subtaskbenglish,subtaskbdanish,subtaskbrussian
443938194715713536,support,true,NaN,NaN
774165935041093633,support,true,NaN,NaN
784071228248109057,support,true,NaN,NaN
784118929799073793,support,true,NaN,NaN
784216706080178176,support,true,NaN,NaN
...,...,...,...,...
8m9y0o,query,true,NaN,NaN
938o5h,support,true,NaN,NaN
ayn70,query,false,NaN,NaN
wfajw,query,true,NaN,NaN


yeah


# Data Preparation

In [8]:
def createtuplelist(structurepath):
    with open(structurepath) as f:
        lines = f.readlines()
    for i in lines:
        print(i,"\n")

    pattern = r'|\"|[[]]|:|[[]|[]]|\\|\''
    regex = re.compile(pattern, flags=re.IGNORECASE)

    lines2 = re.sub(pattern,'',str(lines))
    #print(lines2)

    splitlines = re.split('(\w*.)', lines2)
    splitlines = [i for i in splitlines if i and i != splitlines[0]]
    #print("\nsplitlines:",splitlines,"\n")

    sources =[]
    perspective = ""
    tupleset = []

    for i in range(len(splitlines)-1):
        if not i == 0:
            if i == 1:
                sources.append(splitlines[i][:len(splitlines[i]) -1])

            perspective = splitlines[i][:len(splitlines[i]) -1]
            tupleset.append([sources[-1], perspective])

            if splitlines[i].endswith("{"):
                sources.append(splitlines[i][:len(splitlines[i]) -1])
            if splitlines[i].endswith("}"):
                sources.pop()
            if not perspective:
                tupleset.pop()

    #for i in range(len(tupleset)):
            #print(i,tupleset[i][0],tupleset[i][1])
            
    return(tupleset)

In [9]:
def createtwittertuplelist(structurepath):
    with open(structurejson) as f:
        #print("4")
        lines = f.readlines()
    #for i in lines:
        #print(i,"\n")

    #convert the json with regex to only contain '{' ',' numbers and '}'    
    pattern = r'|\"|[[]]|:|[[]|[]]|\\|\''
    regex = re.compile(pattern, flags=re.IGNORECASE)

    lines2 = re.sub(pattern,'',str(lines))

    #split them by numbers and non-numbers to create a tupleset of the interactions
    splitlines = re.split('(\d*.)', lines2)
    splitlines = [i for i in splitlines if i and i != splitlines[0]]

    sources =[]
    perspective = ""
    tupleset = []

    #the split string are turned into tuple-sets as follows : first number is the claim, 
    #the second is perspective the first set of tuples is always the same
    #number on both sides, to indicate that it's a new thread
#

#
    for i in range(len(splitlines)):
        #print("6")
        if not i == 0:
            #print("7")
            if i == 1:
                #print("8")
                sources.append(splitlines[i][:len(splitlines[i]) -1])
                #print("9")

            perspective = splitlines[i][:len(splitlines[i]) -1]
            tupleset.append([sources[-1], perspective])
            #print("10")

            if splitlines[i].endswith("{"):
                sources.append(splitlines[i][:len(splitlines[i]) -1])
                #print("11")
            if splitlines[i].endswith("}"):
                sources.pop()
                #print("12")
            if not perspective:
                tupleset.pop()
                #print("13")
    return(tupleset)

In [15]:
%%time
# copy of copy to put print between every line
# <<<<<<<RUN THIS

df =pd.DataFrame(
{"Label" : [], 
"x" : [], 
"y" : [],
"claim" : [], 
"perspective" : []})


label = ""
reply = ""
source = ""
unimportantcount = 0
unimportantmarks=[]

newsbookmarks=[0]
threadbookmarks = [0]
pnumber = 0
alltuples = []

#print("0:",pathfolder)
#print("0")
for trainfolders in os.listdir(pathfolder):
    trainfolder = os.path.join(pathfolder, trainfolders) #\threads\en\all the folders
    #print("1")
    #print("1:",trainfolder)
    if "reddit" in trainfolder:
        if os.path.isdir(trainfolder):
            #print("2")
            for topicfolders in os.listdir(trainfolder): 
                allreplies = os.path.join(trainfolder, topicfolders) #\threads\en\all the folders\ threads in number-form
                #print(allreplies)
                #print(allreplies)
                repliesfolder = os.path.join(allreplies, replyfoldername) #\threads in number-form\reactions
                sourcefolder = os.path.join(allreplies, sourcefoldername)
                structurejson = os.path.join(allreplies, structurejsonname)
                #print("3")
    #######            
    ####        
                #print(structurejson)
                tupleset = createtuplelist(structurejson)
                alltuples.append(tupleset) # save all tuplesets to access later for grading

                for i in range(len(tupleset)):
                    #print("14")
                    #print(tupleset,tupleset[i])
                    #############################################
                    if tupleset[i][0] == tupleset[i][1]: # claim = TOPIC, perspective = THREAD
                        #print("14.1")
                        for sources in os.listdir(sourcefolder):
                            #print("14.2")
                            sourcepath = os.path.join(sourcefolder, sources)
                            sourcejson = pd.read_json(sourcepath) 
                            #print("14.3")
                        presource = sourcejson.loc['children','data']
                        normedf = pd.json_normalize(presource)
                        #print("hsh")
                        ####source = os.path.basename(os.path.normpath(trainfolder))
                        #print("14/s")
                        reply = normedf.iloc[0]['data.title']
                        source = reply
                        #print("14/6")
                        #determine label
                        print("annloc:",annotation.loc[tupleset[i][0]]['subtaskaenglish'])
                        if annotation.loc[tupleset[i][0]]['subtaskaenglish'] == pos_support_label: 
                            print("true!")
                            label = "1"
                        elif annotation.loc[tupleset[i][0]]['subtaskaenglish'] == neg_support_label: 
                            print("false!")
                            label = "0"
                        else:
                            print("ireelevant!")
                            label = "0"
                            unimportantmarks.append(len(df.index))



                    else: #claim = THREAD, perspective = TWEET   
                        #print("20")
                        for sources in os.listdir(sourcefolder):
                            sourcepath = os.path.join(sourcefolder, sources)
                            sourcejson = pd.read_json(sourcepath)

                            #print("21")
                        #check if claim in sourcefolder    
                        if tupleset[i][0] in str(sourcejson):
                            #print("[0] in sourcejson", sources)
                            #print("22")
                            presource = sourcejson.loc['children','data']
                            sourcedf = pd.json_normalize(presource)
                            source = sourcedf.iloc[0]['data.title'] #claim


                        #check if claim in repliesfolder
                        else:
                            #print("23")
                            magicnum = str(tupleset[i][0])
                            magicnum +=".json"
                            #print(magicnum)
                            replypath = replypath = os.path.join(repliesfolder,magicnum)                        
                            if os.path.isfile(replypath):
                                #print(replypath,"is correct!")
                                replyjson = pd.read_json(replypath)
                                source = replyjson.loc['body','data']
                            #print("24 but better")


                        #get perspective from the sourcefolder            
                        if tupleset[i][1] in str(sourcejson):
                            #print("26")
                            label = "0"
                            #print("[1] in sourcejson")
                            presource = sourcejson.loc['children','data']
                            sourcedf = pd.json_normalize(presource)
                            source = sourcedf.iloc[0]['data.title'] #claim
                            # get label
                            sourcenumber = tupleset[i][0]
                            if sourcesnumber in annotation.index:
                                #print("27")
                                stance = annotation.loc[sourcesnumber,"subtaskaenglish"]   
                            print("stance",stance)
                            if stance == pos_support_label:
                                #print("28")
                                label = "1"
                            elif stance == neg_support_label:
                                #print("29")
                                label = "0" #label 
                        #get perspective from the repliesfolder
                        else:
                            #print("30")
                            label = "0"

                            magicnum = str(tupleset[i][1])
                            magicnum +=".json"
                            #print(magicnum)
                            replypath = replypath = os.path.join(repliesfolder,magicnum)                        
                            if os.path.isfile(replypath):
                                #print(replypath,"is correct!")
                                replyjson = pd.read_json(replypath)
                                try: 
                                    reply = replyjson.loc['body','data']
                                    #print("reply had body")
                                except:
                                    reply = "0"
                                    unimportantcount += 1
                                    unimportantmarks.append(len(df.index))
                                    #print("reply didnt have body")
                            #print("31 but better")

                            pnumber = tupleset[i][1]
                            if pnumber in annotation.index:
                                #print("32")
                                print("finalstance",annotation.loc[pnumber, 'subtaskaenglish'])
                                if not pd.isnull(annotation.loc[pnumber, 'subtaskaenglish']):
                                    #print("33")
                                    stance =  annotation.loc[pnumber,"subtaskaenglish"]
                                if stance == pos_support_label:
                                    #print("36")
                                    label = "1"
                                elif stance == neg_support_label:
                                    #print("37")
                                    label = "0"
                                else:
                                    #print("38")
                                    label = "0"
                                    unimportantcount +=1
                                    unimportantmarks.append(len(df.index))

                    #print(label, "0", "0", source, "||",reply)
                    df.loc[len(df.index)] = [label, "0", "0", source, reply] ### label=0 for now, bc we don't need it
                    print(df.loc[len(df.index)-1])
                    #print("39")
                    print("<<<<<<<",len(df.index), len(tupleset))
                threadbookmarks.append(len(df.index)-1)
        newsbookmarks.append(len(df.index))

##################################################
###################################################
##################################################

#get the same twitter code from PHEME-code        
    elif "twitter" in trainfolder:
        #print("wop")
        #print(trainfolder)
        if os.path.isdir(trainfolder):
            #print("2")
            for newtrainfolders in os.listdir(trainfolder):
                newtrainfolder = os.path.join(trainfolder,newtrainfolders)
                for topicfolders in os.listdir(newtrainfolder): 
                    allreplies = os.path.join(newtrainfolder, topicfolders) #\threads\en\all the folders\ threads in number-form
                    #print(allreplies)
                    #print(allreplies)
                    repliesfolder = os.path.join(allreplies, replyfoldername) #\threads in number-form\reactions
                    sourcefolder = os.path.join(allreplies, sourcefoldername)
                    structurejson = os.path.join(allreplies, structurejsonname)
                    #print("3")
        #######     
                    #open the tweet structure json
                    tupleset = createtwittertuplelist(structurejson)
                    alltuples.append(tupleset) # save all tuplesets to access later for grading

                    for i in range(len(tupleset)):
                        #print("14")
                        #print(tupleset,tupleset[i])
                        #############################################
                        if tupleset[i][0] == tupleset[i][1]: # claim = TOPIC, perspective = THREAD
                            #print("14.1")
                            for sources in os.listdir(sourcefolder):
                                #print("14.2")
                                sourcepath = os.path.join(sourcefolder, sources)
                                sourcejson = pd.read_json(sourcepath) 
                                #print("14.3")
                            reply = sourcejson.loc['symbols','text']
                            source = os.path.basename(os.path.normpath(trainfolder))
                            #print("14/6")
                            #determine label
                            print("twitanno:",annotation.loc[tupleset[i][0]]['subtaskaenglish'])
                            if annotation.loc[tupleset[i][0]]['subtaskaenglish'] == pos_support_label: 
                                print("tr")
                                label = "1"
                            elif annotation.loc[tupleset[i][0]]['subtaskaenglish'] == neg_support_label: 
                                print("fal")
                                label = "0"
                            else:
                                print("irr")
                                label = "0"
                                unimportantmarks.append(len(df.index))
                            #print("14/3")

                        #############################################
                        else: #claim = THREAD (source-tweet), perspective = TWEET(replies)   
                            #print("20")
                            for sources in os.listdir(sourcefolder):
                                sourcepath = os.path.join(sourcefolder, sources)
                                sourcejson = pd.read_json(sourcepath)
                                #print("21")
                            #check if claim in sourcefolder    
                            if tupleset[i][0] in str(sourcejson):
                                #print("[0] in sourcejson", sources)
                                #print("22")
                                source = sourcejson.loc['symbols','text'] #claim

                            #check if claim in repliesfolder
                            else:
                                #print("23")
                                magicnum = str(tupleset[i][0])
                                magicnum +=".json"
                                #print(magicnum)
                                replypath = replypath = os.path.join(repliesfolder,magicnum)                        
                                if os.path.isfile(replypath):
                                    #print(replypath,"is correct!")
                                    replyjson = pd.read_json(replypath)
                                    source = replyjson.loc['symbols','text']
                                #print("24 but better")


                            #get perspective from the sourcefolder            
                            if tupleset[i][1] in str(sourcejson):
                                #print("26")
                                label = "0"
                                #print("[1] in sourcejson")
                                source = sourcejson.loc['symbols','text'] #claim
                                # get label
                                sourcenumber = tupleset[i][0]
                                if sourcesnumber in annotationkey.index:
                                    #print("27")
                                    stance = annotationkey.loc[sourcesnumber,"support"]   
                                    print("twitstance",stance)
                                if stance == "supporting":
                                    #print("28")
                                    label = "1"
                                elif stance == "denying":
                                    #print("29")
                                    label = "0" #label 
                            #get perspective from the repliesfolder
                            else:
                               # print("30")
                                label = "0"

                                magicnum = str(tupleset[i][1])
                                magicnum +=".json"
                                #print(magicnum)
                                replypath = replypath = os.path.join(repliesfolder,magicnum)                        
                                if os.path.isfile(replypath):
                                    #print(replypath,"is correct!")
                                    replyjson = pd.read_json(replypath)
                                    reply = replyjson.loc['symbols','text']
                                #print("31 but better")

                                pnumber = int(tupleset[i][1])
                                if pnumber in annotation.index:
                                    #print("32")
                                    print("tweetfinalstance",annotation.loc[pnumber, 'subtaskaenglish'])
                                    if not pd.isnull(annotation.loc[pnumber, 'subtaskaenglish']):
                                        #print("33")
                                        stance =  annotation.loc[pnumber,"subtaskaenglish"]
                                    if stance == pos_support_label:
                                        #print("36")
                                        label = "1"
                                    elif stance == neg_support_label:
                                        #print("37")
                                        label = "0"
                                    else:
                                        #print("38")
                                        label = "0"
                                        unimportantcount +=1
                                        unimportantmarks.append(len(df.index))

                        #print(label, "0", "0", source, "||",reply)
                        df.loc[len(df.index)] = [label, "0", "0", source, reply] ### label=0 for now, bc we don't need it
                        print(df.loc[len(df.index)-1])
                        #print("39")
                        print("<<<<<<<",len(df.index), len(tupleset))
                    threadbookmarks.append(len(df.index)-1)
            newsbookmarks.append(len(df.index))

{"1d7lzf":{"c9nxdmq":{"c9nxhfi":{"c9nzmif":{"c9o82ke":{"c9o8xe4":{"c9o8yuo":[]}}},"c9nxjy1":{"c9nxo9o":{"c9nxp8p":[]}}}},"caacp6f":[],"c9oi2sx":[],"c9nu7oo":{"c9nxdwv":{"c9nxu4i":[]},"c9nxfnz":[],"c9nxufe":[]}}}
 

annloc: query
ireelevant!
Label                                                          0
x                                                              0
y                                                              0
claim          Debunk This: A friend of mine claims Red Dye c...
perspective    Debunk This: A friend of mine claims Red Dye c...
Name: 0, dtype: object
<<<<<<< 1 17
finalstance comment
Label                                                          0
x                                                              0
y                                                              0
claim          Debunk This: A friend of mine claims Red Dye c...
perspective    From wikipedia: "Though past research showed n...
Name: 1, dtype: object
<<<<<<< 2 17
finalstance comme

finalstance comment
Label                                                          0
x                                                              0
y                                                              0
claim          Aren't the companies dodging state taxes as well?
perspective    Definitely but there are generally fewer deduc...
Name: 24, dtype: object
<<<<<<< 25 10
finalstance comment
Label                                                          0
x                                                              0
y                                                              0
claim          It's not a debunking, but it's worth noting th...
perspective    Yep and education spending is closer to $900 b...
Name: 25, dtype: object
<<<<<<< 26 10
finalstance deny
Label                                                          0
x                                                              0
y                                                              0
claim          It's no

finalstance support
Label                                                          1
x                                                              0
y                                                              0
claim                                      It's not. See below. 
perspective    Pretty sure that's true for most main stream m...
Name: 52, dtype: object
<<<<<<< 53 20
finalstance comment
Label                                                          0
x                                                              0
y                                                              0
claim                                      It's not. See below. 
perspective    I mean I thought this was already a known thin...
Name: 53, dtype: object
<<<<<<< 54 20
finalstance comment
Label                                                        0
x                                                            0
y                                                            0
claim                    

finalstance comment
Label                                                          0
x                                                              0
y                                                              0
claim          Ummm. Mitochondria suggests *homo sapiens sapi...
perspective                                            [removed]
Name: 76, dtype: object
<<<<<<< 77 19
finalstance comment
Label                  0
x                      0
y                      0
claim          [removed]
perspective    [removed]
Name: 77, dtype: object
<<<<<<< 78 19
finalstance comment
Label                  0
x                      0
y                      0
claim          [removed]
perspective    [removed]
Name: 78, dtype: object
<<<<<<< 79 19
finalstance comment
Label                                                          0
x                                                              0
y                                                              0
claim          Our species is very y

finalstance comment
Label                                                          0
x                                                              0
y                                                              0
claim          Here's the Wikipedia take on the disaster: htt...
perspective    Non-Mobile link: https://en.wikipedia.org/wiki...
Name: 103, dtype: object
<<<<<<< 104 7
finalstance comment
Label                                                          0
x                                                              0
y                                                              0
claim          Here's the Wikipedia take on the disaster: htt...
perspective    &gt; anti-McCain propaganda.\n\nDoled out beca...
Name: 104, dtype: object
<<<<<<< 105 7
finalstance comment
Label                                                          0
x                                                              0
y                                                              0
claim          He

...

Label                                                          0
x                                                              0
y                                                              0
claim          WTF RT @Jeggit: Believe it or not, this is a s...
perspective    @jimmyvasser @Jeggit This picture has been goi...
Name: 1722, dtype: object
<<<<<<< 1723 34
Label                                                          0
x                                                              0
y                                                              0
claim          WTF RT @Jeggit: Believe it or not, this is a s...
perspective    @jimmyvasser @Jeggit At least he is passing in...
Name: 1723, dtype: object
<<<<<<< 1724 34
Label                                                          0
x                                                              0
y                                                              0
claim          WTF RT @Jeggit: Believe it or not, this is a s...
perspe

Label                                                          0
x                                                              0
y                                                              0
claim          WTF RT @Jeggit: Believe it or not, this is a s...
perspective          @jimmyvasser @Jeggit @DopeMohammed told you
Name: 1745, dtype: object
<<<<<<< 1746 34
Label                                                          0
x                                                              0
y                                                              0
claim          WTF RT @Jeggit: Believe it or not, this is a s...
perspective    @jimmyvasser @Jeggit This is so fake. Seen thi...
Name: 1746, dtype: object
<<<<<<< 1747 34
Label                                                          0
x                                                              0
y                                                              0
claim          WTF RT @Jeggit: Believe it or not, this is a s...
perspe

Label                                                          0
x                                                              0
y                                                              0
claim          BREAKING: Texas Mosque Refuses To Help Refugee...
perspective    @KristinaSpark17 We all need to remember this,...
Name: 1770, dtype: object
<<<<<<< 1771 6
Label                                                          0
x                                                              0
y                                                              0
claim          BREAKING: Texas Mosque Refuses To Help Refugee...
perspective    @KristinaSpark17 While Jewish summer camps are...
Name: 1771, dtype: object
<<<<<<< 1772 6
Label                                                          0
x                                                              0
y                                                              0
claim          BREAKING: Texas Mosque Refuses To Help Refugee...
perspect

Label                                                          0
x                                                              0
y                                                              0
claim          Virgin Islands Allows National Guard To Seize ...
perspective    @RhondaSexton17 @gal_deplorable You are so rig...
Name: 1795, dtype: object
<<<<<<< 1796 8
Label                                                          0
x                                                              0
y                                                              0
claim          Virgin Islands Allows National Guard To Seize ...
perspective    @America3675 @gal_deplorable So they take all ...
Name: 1796, dtype: object
<<<<<<< 1797 8
Label                                                          0
x                                                              0
y                                                              0
claim          Virgin Islands Allows National Guard To Seize ...
perspect

Label                                                          0
x                                                              0
y                                                              0
claim          @KellyannePolls Know what else is heartbreakin...
perspective    @oldwolftj1 @politicalmablog @KellyannePolls H...
Name: 1818, dtype: object
<<<<<<< 1819 22
Label                                                          0
x                                                              0
y                                                              0
claim          @KellyannePolls Know what else is heartbreakin...
perspective    @DonaldRayBritt2 @KellyannePolls These two thi...
Name: 1819, dtype: object
<<<<<<< 1820 22
Label                                                          0
x                                                              0
y                                                              0
claim          @KellyannePolls Know what else is heartbreakin...
perspe

In [16]:
df

,Label,x,y,claim,perspective
0,0,0,0,Debunk This: A friend of mine claims Red Dye c...,Debunk This: A friend of mine claims Red Dye c...
1,0,0,0,Debunk This: A friend of mine claims Red Dye c...,"From wikipedia: ""Though past research showed n..."
2,0,0,0,"From wikipedia: ""Though past research showed n...","Hah, well I'll be damned. \n\nAnd to be fair, ..."
3,0,0,0,"Hah, well I'll be damned. \n\nAnd to be fair, ...","Well, to be fair as well, it's Wikipedia.\n\nT..."
4,0,0,0,"Well, to be fair as well, it's Wikipedia.\n\nT...",Red-head here. Food dyes are often used in shi...
...,...,...,...,...,...
1826,0,0,0,"From @JimPuzzanghera:\n\n""The House Republican...","From @MartinWisckol:\n\n""Future victims of wil..."
1827,0,0,0,"From @JimPuzzanghera:\n\n""The House Republican...","From @clairezillman:\n\n""Those fires took plac..."
1828,0,0,0,"From @JimPuzzanghera:\n\n""The House Republican...","And @SteveKnight25, your ""YEA"" vote in favor o..."
1829,0,0,0,"From @JimPuzzanghera:\n\n""The House Republican...",@CA25UP @SteveKnight25 DAAAAAAMN. You got ownn...


In [17]:
# regex: if '@' or 'htttp://', then delete whole string/ until theres a blank space
pattern = r'@|#|http(\S)*'
regex = re.compile(pattern, flags=re.IGNORECASE)

for i in range((len(df))):
    if (type(df['perspective'][i]) != float):
        df['perspective'][i] = regex.sub('', df['perspective'][i])
    if (type(df['claim'][i]) != float):
        df['claim'][i] = regex.sub('', df['claim'][i])

In [18]:
df

,Label,x,y,claim,perspective
0,0,0,0,Debunk This: A friend of mine claims Red Dye c...,Debunk This: A friend of mine claims Red Dye c...
1,0,0,0,Debunk This: A friend of mine claims Red Dye c...,"From wikipedia: ""Though past research showed n..."
2,0,0,0,"From wikipedia: ""Though past research showed n...","Hah, well I'll be damned. \n\nAnd to be fair, ..."
3,0,0,0,"Hah, well I'll be damned. \n\nAnd to be fair, ...","Well, to be fair as well, it's Wikipedia.\n\nT..."
4,0,0,0,"Well, to be fair as well, it's Wikipedia.\n\nT...",Red-head here. Food dyes are often used in shi...
...,...,...,...,...,...
1826,0,0,0,"From JimPuzzanghera:\n\n""The House Republican ...","From MartinWisckol:\n\n""Future victims of wild..."
1827,0,0,0,"From JimPuzzanghera:\n\n""The House Republican ...","From clairezillman:\n\n""Those fires took place..."
1828,0,0,0,"From JimPuzzanghera:\n\n""The House Republican ...","And SteveKnight25, your ""YEA"" vote in favor of..."
1829,0,0,0,"From JimPuzzanghera:\n\n""The House Republican ...",CA25UP SteveKnight25 DAAAAAAMN. You got ownnnn...


In [19]:
df = df.reset_index(drop=True)
df

,Label,x,y,claim,perspective
0,0,0,0,Debunk This: A friend of mine claims Red Dye c...,Debunk This: A friend of mine claims Red Dye c...
1,0,0,0,Debunk This: A friend of mine claims Red Dye c...,"From wikipedia: ""Though past research showed n..."
2,0,0,0,"From wikipedia: ""Though past research showed n...","Hah, well I'll be damned. \n\nAnd to be fair, ..."
3,0,0,0,"Hah, well I'll be damned. \n\nAnd to be fair, ...","Well, to be fair as well, it's Wikipedia.\n\nT..."
4,0,0,0,"Well, to be fair as well, it's Wikipedia.\n\nT...",Red-head here. Food dyes are often used in shi...
...,...,...,...,...,...
1826,0,0,0,"From JimPuzzanghera:\n\n""The House Republican ...","From MartinWisckol:\n\n""Future victims of wild..."
1827,0,0,0,"From JimPuzzanghera:\n\n""The House Republican ...","From clairezillman:\n\n""Those fires took place..."
1828,0,0,0,"From JimPuzzanghera:\n\n""The House Republican ...","And SteveKnight25, your ""YEA"" vote in favor of..."
1829,0,0,0,"From JimPuzzanghera:\n\n""The House Republican ...",CA25UP SteveKnight25 DAAAAAAMN. You got ownnnn...


In [33]:
#export to csv/tsv
df.to_csv(export_destination, sep = '\t', index = False)

In [34]:
unimportantcount

786

# Tribrid_pos for stance classificaiton

In [52]:
tribrid_results = pd.read_csv(trib_results)

In [53]:
tribrid_results

,undermine_score,support_score,predict_label,gold
0,-5.970719,4.975022,1,0
1,-4.017447,2.925926,1,0
2,-2.983997,2.623589,1,0
3,-2.822363,3.297756,1,0
4,-3.973253,3.352272,1,0
...,...,...,...,...
1826,-1.083951,0.421390,1,0
1827,-4.329742,3.757640,1,0
1828,0.657087,-1.052095,0,0
1829,-1.782537,1.125543,1,0


In [54]:
#change the 0's to -1's to better calculate
for i in range(len(tribrid_results)):
    if tribrid_results['predict_label'][i] == 0:
        tribrid_results.at[i,'predict_label'] = -1
    if tribrid_results['gold'][i] == 0:
        tribrid_results.at[i,'gold'] = -1
        
    # if the row is unimportant, then mark it as 0
    #if i in unimportantmarks:
        #tribrid_results.at[i,'gold'] = 0

In [55]:
tribrid_results

,undermine_score,support_score,predict_label,gold
0,-5.970719,4.975022,1,-1
1,-4.017447,2.925926,1,-1
2,-2.983997,2.623589,1,-1
3,-2.822363,3.297756,1,-1
4,-3.973253,3.352272,1,-1
...,...,...,...,...
1826,-1.083951,0.421390,1,-1
1827,-4.329742,3.757640,1,-1
1828,0.657087,-1.052095,-1,-1
1829,-1.782537,1.125543,1,-1


In [56]:
print(len(alltuples))       #tuples of numbers of reactions in tuples, 1 tuple/entry is for a whole thread
print(len(threadbookmarks)) #there are 298/297 threads
print(threadbookmarks[1])   #its the 7, [0] is 0
#print(tribrid_results)

for i in range(len(tupleset)):
    print(i,tupleset[i][0],tupleset[i][1])
print(len(alltuples[0]))

print(tribrid_results.iloc[threadbookmarks[2]:(threadbookmarks[3]+1)])

#0's end = sourcenumber/ list[0]
#look at first number
#  if sourcenumber, make list[i] = number2, predict_label (-1 or 1)
#  if else, check through list with same number, make list[i] = number2, predict_label(number1) * predict_label(number2)
#calculate all numbers together

81
82
16
0 951976286301204480 951976286301204480
1 951976286301204480 952070086621200384
2 951976286301204480 951977378753818624
3 951976286301204480 951976715357573121
4 951976286301204480 951978933758574592
5 951976286301204480 951979616100524032
6 951976286301204480 951982101485809665
7 951976286301204480 952002837243822080
17
    undermine_score  support_score  predict_label  gold
26        -3.723805       6.188354              1    -1
27        -5.257926       4.548798              1     1
28        -5.052714       4.388851              1    -1
29        -1.504977       0.148183              1    -1
30        -1.175895       0.901860              1    -1
31        -0.580611       0.394767              1     1


In [57]:
threadbookmarks2 = []
[threadbookmarks2.append(x) for x in threadbookmarks if x not in threadbookmarks2]
#print(threadbookmarks2)
#print(len(threadbookmarks2))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [58]:
flattuples = []
for i in range(len(alltuples)): #Traversing through the main list
  for j in range (len(alltuples[i])): #Traversing through each sublist
    flattuples.append(alltuples[i][j]) #Appending elements into our flat_list

newdf3 = tribrid_results.copy()
newdf3.rename(columns = {'undermine_score' : 'claim', 'support_score' : 'perspective', 'gold' : 'result'}, inplace = True)
newdf3 = newdf3.astype({"claim": str, "perspective": str})
for i in range(len(newdf3)):
    newdf3.at[i,'claim'] = flattuples[i][0][-5:]
    newdf3.at[i,'perspective'] = flattuples[i][1][-5:]
    #changing all values to thread/replynumbers

    if newdf3.at[i,'claim'] == newdf3.at[i,'perspective']:
        newdf3.at[i,'predict_label'] = newdf3.at[i,'result']
    #changing all thread "predictions" with results, as it is our guide to calculate how right/wrong people are
    
    else:
        claim_result = newdf3.index[newdf3['perspective'] == newdf3.at[i,'claim']]
        #find the claim's stance by getting the index(es)(but then getting the first instance of the index) 
        #of the claim's number in perspective column
        newdf3.at[i,'result'] = newdf3.at[claim_result[0],'result'] * newdf3.at[i,'predict_label']
    #changing the result with previous stance * this stance (written better in evernote)
print("len of df3:", len(newdf3))    
for i in range(len(newdf3)):
    print(newdf3.iloc[i],"\n")
#newdf3

len of df3: 1831
claim            d7lzf
perspective      d7lzf
predict_label       -1
result              -1
Name: 0, dtype: object 

claim            d7lzf
perspective      nxdmq
predict_label        1
result              -1
Name: 1, dtype: object 

claim            nxdmq
perspective      nxhfi
predict_label        1
result              -1
Name: 2, dtype: object 

claim            nxhfi
perspective      nzmif
predict_label        1
result              -1
Name: 3, dtype: object 

claim            nzmif
perspective      o82ke
predict_label        1
result              -1
Name: 4, dtype: object 

claim            o82ke
perspective      o8xe4
predict_label        1
result              -1
Name: 5, dtype: object 

claim            o8xe4
perspective      o8yuo
predict_label        1
result              -1
Name: 6, dtype: object 

claim            nxhfi
perspective      nxjy1
predict_label        1
result              -1
Name: 7, dtype: object 

claim            nxjy1
perspective      nxo9o
p

Name: 115, dtype: object 

claim            vvv51
perspective      5at6i
predict_label        1
result              -1
Name: 116, dtype: object 

claim            5at6i
perspective      9ff9b
predict_label        1
result              -1
Name: 117, dtype: object 

claim            vvv51
perspective      5tfdl
predict_label       -1
result               1
Name: 118, dtype: object 

claim            vvv51
perspective      w69jf
predict_label        1
result              -1
Name: 119, dtype: object 

claim            vvv51
perspective      5vb5a
predict_label        1
result              -1
Name: 120, dtype: object 

claim            vvv51
perspective      5j6c7
predict_label       -1
result               1
Name: 121, dtype: object 

claim            5j6c7
perspective      5nly3
predict_label       -1
result              -1
Name: 122, dtype: object 

claim            vvv51
perspective      5j6fn
predict_label        1
result              -1
Name: 123, dtype: object 

claim            vvv5

Name: 229, dtype: object 

claim            tvotl
perspective      txyah
predict_label        1
result              -1
Name: 230, dtype: object 

claim            hrc7n
perspective      tjn4d
predict_label        1
result              -1
Name: 231, dtype: object 

claim            tjn4d
perspective      tkfur
predict_label        1
result              -1
Name: 232, dtype: object 

claim            tkfur
perspective      ufige
predict_label        1
result              -1
Name: 233, dtype: object 

claim            hrc7n
perspective      taozd
predict_label       -1
result               1
Name: 234, dtype: object 

claim            hrc7n
perspective      tq0bk
predict_label        1
result              -1
Name: 235, dtype: object 

claim            hrc7n
perspective      tyen0
predict_label        1
result              -1
Name: 236, dtype: object 

claim            hrc7n
perspective      tnp8r
predict_label        1
result              -1
Name: 237, dtype: object 

claim            tnp8

Name: 315, dtype: object 

claim            zztcn
perspective      0ktiq
predict_label        1
result               1
Name: 316, dtype: object 

claim            zx7ao
perspective      zzafj
predict_label        1
result               1
Name: 317, dtype: object 

claim            zzafj
perspective      zzvu6
predict_label        1
result               1
Name: 318, dtype: object 

claim            zzvu6
perspective      01w0y
predict_label        1
result               1
Name: 319, dtype: object 

claim            01w0y
perspective      03bq0
predict_label        1
result               1
Name: 320, dtype: object 

claim            03bq0
perspective      06ko6
predict_label        1
result               1
Name: 321, dtype: object 

claim            03bq0
perspective      03off
predict_label        1
result               1
Name: 322, dtype: object 

claim            03bq0
perspective      040an
predict_label        1
result               1
Name: 323, dtype: object 

claim            040a

Name: 522, dtype: object 

claim            79j1c
perspective      b763l
predict_label        1
result              -1
Name: 523, dtype: object 

claim            e86kt
perspective      e86kt
predict_label       -1
result              -1
Name: 524, dtype: object 

claim            e86kt
perspective      t5qmg
predict_label        1
result              -1
Name: 525, dtype: object 

claim            t5qmg
perspective      t5vqk
predict_label       -1
result               1
Name: 526, dtype: object 

claim            t5vqk
perspective      t6p6c
predict_label       -1
result              -1
Name: 527, dtype: object 

claim            e86kt
perspective      t5u4s
predict_label        1
result              -1
Name: 528, dtype: object 

claim            e86kt
perspective      t6wqj
predict_label       -1
result               1
Name: 529, dtype: object 

claim            t6wqj
perspective      t7a4g
predict_label       -1
result              -1
Name: 530, dtype: object 

claim            lkwd

...

Name: 1235, dtype: object 

claim            94400
perspective      81408
predict_label        1
result               1
Name: 1236, dtype: object 

claim            13344
perspective      13344
predict_label        1
result               1
Name: 1237, dtype: object 

claim            13344
perspective      36257
predict_label        1
result               1
Name: 1238, dtype: object 

claim            13344
perspective      75873
predict_label        1
result               1
Name: 1239, dtype: object 

claim            13344
perspective      47745
predict_label        1
result               1
Name: 1240, dtype: object 

claim            13344
perspective      63937
predict_label        1
result               1
Name: 1241, dtype: object 

claim            79329
perspective      79329
predict_label        1
result               1
Name: 1242, dtype: object 

claim            79329
perspective      22080
predict_label       -1
result              -1
Name: 1243, dtype: object 

claim       

Name: 1369, dtype: object 

claim            14177
perspective      15616
predict_label        1
result               1
Name: 1370, dtype: object 

claim            14177
perspective      24192
predict_label        1
result               1
Name: 1371, dtype: object 

claim            14177
perspective      44320
predict_label       -1
result              -1
Name: 1372, dtype: object 

claim            14177
perspective      03841
predict_label        1
result               1
Name: 1373, dtype: object 

claim            14177
perspective      84672
predict_label        1
result               1
Name: 1374, dtype: object 

claim            14177
perspective      51873
predict_label        1
result               1
Name: 1375, dtype: object 

claim            14177
perspective      74721
predict_label        1
result               1
Name: 1376, dtype: object 

claim            14177
perspective      54145
predict_label        1
result               1
Name: 1377, dtype: object 

claim       

Name: 1480, dtype: object 

claim            23904
perspective      05216
predict_label        1
result               1
Name: 1481, dtype: object 

claim            23904
perspective      94146
predict_label        1
result               1
Name: 1482, dtype: object 

claim            23904
perspective      07296
predict_label        1
result               1
Name: 1483, dtype: object 

claim            23904
perspective      45637
predict_label        1
result               1
Name: 1484, dtype: object 

claim            23904
perspective      90752
predict_label        1
result               1
Name: 1485, dtype: object 

claim            23904
perspective      09920
predict_label        1
result               1
Name: 1486, dtype: object 

claim            23904
perspective      20960
predict_label        1
result               1
Name: 1487, dtype: object 

claim            23904
perspective      73536
predict_label        1
result               1
Name: 1488, dtype: object 

claim       


claim            41857
perspective      96864
predict_label        1
result               1
Name: 1614, dtype: object 

claim            41857
perspective      99074
predict_label        1
result               1
Name: 1615, dtype: object 

claim            41857
perspective      52544
predict_label        1
result               1
Name: 1616, dtype: object 

claim            59616
perspective      59616
predict_label        1
result               1
Name: 1617, dtype: object 

claim            59616
perspective      38689
predict_label        1
result               1
Name: 1618, dtype: object 

claim            59616
perspective      45280
predict_label        1
result               1
Name: 1619, dtype: object 

claim            59616
perspective      58981
predict_label        1
result               1
Name: 1620, dtype: object 

claim            59616
perspective      93442
predict_label       -1
result              -1
Name: 1621, dtype: object 

claim            59616
perspective     

Name: 1732, dtype: object 

claim            21633
perspective      05224
predict_label        1
result               1
Name: 1733, dtype: object 

claim            21633
perspective      19713
predict_label        1
result               1
Name: 1734, dtype: object 

claim            21633
perspective      41600
predict_label       -1
result              -1
Name: 1735, dtype: object 

claim            21633
perspective      23680
predict_label        1
result               1
Name: 1736, dtype: object 

claim            21633
perspective      37825
predict_label        1
result               1
Name: 1737, dtype: object 

claim            21633
perspective      47682
predict_label        1
result               1
Name: 1738, dtype: object 

claim            21633
perspective      75744
predict_label        1
result               1
Name: 1739, dtype: object 

claim            21633
perspective      31776
predict_label        1
result               1
Name: 1740, dtype: object 

claim       

In [59]:
for i in range((len(newdf3))):
    if newdf3.at[i,'claim'] == newdf3.at[i,'perspective'] and i in unimportantmarks:
        print("yes!")
        newdf3.at[i,'predict_label'] = 0
        newdf3.at[i,'result'] = 0
        print(newdf3.iloc[i])

yes!
claim            d7lzf
perspective      d7lzf
predict_label        0
result               0
Name: 0, dtype: object
yes!
claim            uniwn
perspective      uniwn
predict_label        0
result               0
Name: 17, dtype: object
yes!
claim            xqilx
perspective      xqilx
predict_label        0
result               0
Name: 41, dtype: object
yes!
claim            eo2zi
perspective      eo2zi
predict_label        0
result               0
Name: 46, dtype: object
yes!
claim            ycnf5
perspective      ycnf5
predict_label        0
result               0
Name: 66, dtype: object
yes!
claim            3czp3
perspective      3czp3
predict_label        0
result               0
Name: 85, dtype: object
yes!
claim            tk0zx
perspective      tk0zx
predict_label        0
result               0
Name: 100, dtype: object
yes!
claim            vvv51
perspective      vvv51
predict_label        0
result               0
Name: 107, dtype: object
yes!
claim            yp0yt
per

In [60]:
#separate reply point system by thread 
elist = []
telist = []
copyforllist = []
llist = []
for i in range(len(newdf3)):
    if newdf3.at[i,'claim'] == newdf3.at[i,'perspective']: #if these numbers are the same, then we have a new thread        
        if i == 0:
            a =(newdf3.at[i,'claim']) #add threadnumber
            telist.append(newdf3.at[i,'result']) #append value to a list
            
        elif i in newsbookmarks:
            aa = tuple((a,telist[:])) #add threadnumber and list with its values to final list
            elist.append(aa)          #add threadnumber and list with its values to final list
            copyforllist.append(aa)
            llist.append(copyforllist[:])
            telist.clear()            #clear temporary list
            copyforllist.clear()
            a =(newdf3.at[i,'claim']) #add threadnumber
            telist.append(newdf3.at[i,'result']) #append value to a list 
            
        else:
            aa = tuple((a,telist[:])) #add threadnumber and list with its values to final list
            elist.append(aa)          #add threadnumber and list with its values to final list
            copyforllist.append(aa)
            telist.clear()            #clear temporary list
            a =(newdf3.at[i,'claim']) #add threadnumber
            telist.append(newdf3.at[i,'result']) #append value to a list
            
    else:
        telist.append(newdf3.at[i,'result']) #append value to a list
aa = tuple((a,telist[:]))
elist.append(aa)
copyforllist.append(aa)
llist.append(copyforllist)
#del elist[0]
#print(len(elist))
#print(elist)

In [61]:
tuplelist = []
for i in range(len(elist)): # [1,-1,-1,-1,1,1,1
    tuple1=tuple((elist[i][0],elist[i][1][0],elist[i][1][1:len(elist)]))
    tuplelist.append(tuple1)

    #divide the tuple into the following:
    #threadnumber, result(if news is true/false), reply-values(1,-1,1.. etc)
    
#for i in range(len(tuplelist)):
#    print(i,tuplelist[i])

In [62]:
counttuple = []
forc = 0
against = 0

df5 =pd.DataFrame(
{"threadnumber" : [],
"news" : [], 
"for" : [], 
"against" : [],
"for%" : [], 
"against%" : [], 
"verdict" : []})

for i in tuplelist:
    #print(i[2])
    for j in range(len(i[2])):
        if (i[2][j] == 1):
            forc += 1
        else:
            against += 1
        #print(i[1][j])
        
    #print(forc,against)
    
    forp = forc/(forc+against)
    againstp = against/(forc+against)
    summed = sum(i[2])
    
    if i[1] > 0 and summed > 0:
        verd = "TP"   #pregnant woman is pregnant
    elif i[1] > 0 and summed < 0:
        verd = "FP"   #pregnant woman is told shes not preg
    elif i[1] < 0 and summed > 0:
        verd = "FN"   #dentist says man is pregnant
    elif i[1] < 0 and summed < 0:
        verd = "TN"   #man is told he is NOT pregnant
    elif i[1] > 0 and summed == 0:
        verd = "UP"   #pregnant woman is by equal amount that she is and is not pregnant
    elif i[1] < 0 and summed == 0:
        verd = "UN"   #man is by equal amount that she is and is not pregnant
    elif i[1] == 0 and summed < 0:
        verd = "TQ"   #true query/comment
    elif i[1] == 0 and summed > 1:
        verd = "FQ"   #false query/comment
    elif i[1] == 0 and summed == 0:
        verd = "UQ"
        
    df5.loc[len(df5.index)] = [i[0], i[1], forc, against, forp, againstp, verd]   
    #print(len(df5))
    forc = 0
    against = 0
df5

,threadnumber,news,for,against,for%,against%,verdict
0,d7lzf,0.0,5.0,11.0,0.312500,0.687500,TQ
1,uniwn,0.0,4.0,5.0,0.444444,0.555556,TQ
2,x8zke,1.0,4.0,0.0,1.000000,0.000000,TP
3,e7pye,-1.0,2.0,6.0,0.250000,0.750000,TN
4,xqilx,0.0,1.0,3.0,0.250000,0.750000,TQ
...,...,...,...,...,...,...,...
76,34624,1.0,1.0,2.0,0.333333,0.666667,FP
77,16128,1.0,5.0,5.0,0.500000,0.500000,UP
78,49345,1.0,6.0,1.0,0.857143,0.142857,TP
79,44576,1.0,8.0,13.0,0.380952,0.619048,FP


In [63]:
reslist = []
df6 = df5.copy()
print(df6)
for i in range(len(llist)):
    print(i, len(llist[i]))
    reslist.append(df6.iloc[0:len(llist[i])])
    df6 = df6.iloc[len(llist[i]):,:]
    print("ln",len(df6))
print(reslist)

   threadnumber  news  for  against      for%  against% verdict
0         d7lzf   0.0  5.0     11.0  0.312500  0.687500      TQ
1         uniwn   0.0  4.0      5.0  0.444444  0.555556      TQ
2         x8zke   1.0  4.0      0.0  1.000000  0.000000      TP
3         e7pye  -1.0  2.0      6.0  0.250000  0.750000      TN
4         xqilx   0.0  1.0      3.0  0.250000  0.750000      TQ
..          ...   ...  ...      ...       ...       ...     ...
76        34624   1.0  1.0      2.0  0.333333  0.666667      FP
77        16128   1.0  5.0      5.0  0.500000  0.500000      UP
78        49345   1.0  6.0      1.0  0.857143  0.142857      TP
79        44576   1.0  8.0     13.0  0.380952  0.619048      FP
80        04480   1.0  6.0      1.0  0.857143  0.142857      TP

[81 rows x 7 columns]
0 25
ln 56
1 56
ln 0
[   threadnumber  news   for  against      for%  against% verdict
0         d7lzf   0.0   5.0     11.0  0.312500  0.687500      TQ
1         uniwn   0.0   4.0      5.0  0.444444  0.555556 

In [64]:
print(reslist[1])

   threadnumber  news   for  against      for%  against% verdict
25        01985   1.0  10.0     16.0  0.384615  0.615385      FP
26        13536   1.0   4.0      7.0  0.363636  0.636364      FP
27        93633   1.0  11.0      8.0  0.578947  0.421053      TP
28        87232   1.0   5.0      2.0  0.714286  0.285714      TP
29        65152   0.0   0.0      5.0  0.000000  1.000000      TQ
..          ...   ...   ...      ...       ...       ...     ...
76        34624   1.0   1.0      2.0  0.333333  0.666667      FP
77        16128   1.0   5.0      5.0  0.500000  0.500000      UP
78        49345   1.0   6.0      1.0  0.857143  0.142857      TP
79        44576   1.0   8.0     13.0  0.380952  0.619048      FP
80        04480   1.0   6.0      1.0  0.857143  0.142857      TP

[56 rows x 7 columns]


In [65]:
index = 0

allresulttables = []

for i in reslist:
    resulttable = pd.DataFrame(
    {"thread: true" : [0,0,0],
     "thread: false" : [0,0,0],
     "thread: query/comment" : [0,0,0]}, 
    index = ["opinion: true", "opinion: false", "opinion: undecided"])
    
    TP = (i.verdict == 'TP').sum()
    FP = (i.verdict == 'FP').sum()
    TN = (i.verdict == 'TN').sum()
    FN = (i.verdict == 'FN').sum()
    UP = (i.verdict == 'UP').sum()
    UN = (i.verdict == 'UN').sum()
    TQ = (i.verdict == 'TQ').sum()
    FQ = (i.verdict == 'FQ').sum()
    UQ = (i.verdict == 'UQ').sum()
    
    resulttable.iloc[0][0] = TP
    resulttable.iloc[1][0] = FP
    resulttable.iloc[2][0] = UP
    resulttable.iloc[0][1] = TN
    resulttable.iloc[1][1] = FN
    resulttable.iloc[2][1] = UN
    resulttable.iloc[0][2] = TQ
    resulttable.iloc[1][2] = FQ
    resulttable.iloc[2][2] = UQ    
        
    allresulttables.append(resulttable)
allresulttables   

[                    thread: true  thread: false  thread: query/comment
 opinion: true                  2              1                     16
 opinion: false                 1              0                      4
 opinion: undecided             0              0                      1,
                     thread: true  thread: false  thread: query/comment
 opinion: true                 42              0                      4
 opinion: false                 7              0                      1
 opinion: undecided             2              0                      0]

In [66]:
for i in allresulttables:
    display(i)

,thread: true,thread: false,thread: query/comment
opinion: true,2,1,16
opinion: false,1,0,4
opinion: undecided,0,0,1


,thread: true,thread: false,thread: query/comment
opinion: true,42,0,4
opinion: false,7,0,1
opinion: undecided,2,0,0


In [67]:
index = 0

allpercentresulttables = []

for i in reslist:
    resulttable = pd.DataFrame(
    {"thread: true" : [0,0,0],
     "thread: false" : [0,0,0],
     "thread: query/comment" : [0,0,0]}, 
    index = ["opinion: true", "opinion: false", "opinion: undecided"])
    
    TP = ((i.verdict == 'TP').sum() * 100.)/len(i)
    FP = ((i.verdict == 'FP').sum() * 100.)/len(i)
    TN = ((i.verdict == 'TN').sum() * 100.)/len(i)
    FN = ((i.verdict == 'FN').sum() * 100.)/len(i)
    UP = ((i.verdict == 'UP').sum() * 100.)/len(i)
    UN = ((i.verdict == 'UN').sum() * 100.)/len(i)
    TQ = ((i.verdict == 'TQ').sum() * 100.)/len(i)
    FQ = ((i.verdict == 'FQ').sum() * 100.)/len(i)
    UQ = ((i.verdict == 'UQ').sum() * 100.)/len(i)
    
    resulttable.iloc[0][0] = TP
    resulttable.iloc[1][0] = FP
    resulttable.iloc[2][0] = UP
    resulttable.iloc[0][1] = TN
    resulttable.iloc[1][1] = FN
    resulttable.iloc[2][1] = UN
    resulttable.iloc[0][2] = TQ
    resulttable.iloc[1][2] = FQ
    resulttable.iloc[2][2] = UQ    
        
    allpercentresulttables.append(resulttable)
allpercentresulttables   

[                    thread: true  thread: false  thread: query/comment
 opinion: true                  8              4                     64
 opinion: false                 4              0                     16
 opinion: undecided             0              0                      4,
                     thread: true  thread: false  thread: query/comment
 opinion: true                 75              0                      7
 opinion: false                12              0                      1
 opinion: undecided             3              0                      0]

In [68]:
for i in allpercentresulttables:
    display(i)

,thread: true,thread: false,thread: query/comment
opinion: true,8,4,64
opinion: false,4,0,16
opinion: undecided,0,0,4


,thread: true,thread: false,thread: query/comment
opinion: true,75,0,7
opinion: false,12,0,1
opinion: undecided,3,0,0


In [69]:
# # todo
# #
#X# do tables percentage wise
# # clean up
# # get the same shit done for the other dataset

In [70]:
index = 0

alltogetherresulttables = []

resulttable = pd.DataFrame(
    {"thread: true" : [0,0,0],
     "thread: false" : [0,0,0],
     "thread: query/comment" : [0,0,0]}, 
    index = ["opinion: true", "opinion: false", "opinion: undecided"])

TP = 0
FP = 0
TN = 0
FN = 0
UP = 0
UN = 0
TQ = 0
FQ = 0
UQ = 0
    
for i in reslist:    
    TP = TP+(i.verdict == 'TP').sum()
    FP = FP+(i.verdict == 'FP').sum()
    TN = TN+(i.verdict == 'TN').sum()
    FN = FN+(i.verdict == 'FN').sum()
    UP = UP+(i.verdict == 'UP').sum()
    UN = UN+(i.verdict == 'UN').sum()
    TQ = TQ+(i.verdict == 'TQ').sum()
    FQ = FQ+(i.verdict == 'FQ').sum()
    UQ = UQ+(i.verdict == 'UQ').sum()
    
    resulttable.iloc[0][0] = TP
    resulttable.iloc[1][0] = FP
    resulttable.iloc[2][0] = UP
    resulttable.iloc[0][1] = TN
    resulttable.iloc[1][1] = FN
    resulttable.iloc[2][1] = UN
    resulttable.iloc[0][2] = TQ
    resulttable.iloc[1][2] = FQ
    resulttable.iloc[2][2] = UQ    
        
alltogetherresulttables.append(resulttable)
display(alltogetherresulttables[0])

,thread: true,thread: false,thread: query/comment
opinion: true,44,1,20
opinion: false,8,0,5
opinion: undecided,2,0,1


In [71]:
index = 0

alltogetherpercentresulttables = []

resulttable = pd.DataFrame(
    {"thread: true" : [0,0,0],
     "thread: false" : [0,0,0],
     "thread: query/comment" : [0,0,0]}, 
    index = ["opinion: true", "opinion: false", "opinion: undecided"])

TP = 0
FP = 0
TN = 0
FN = 0
UP = 0
UN = 0
TQ = 0
FQ = 0
UQ = 0
    
for i in reslist:    
    TP = ((i.verdict == 'TP').sum() * 100.)/len(i)
    FP = ((i.verdict == 'FP').sum() * 100.)/len(i)
    TN = ((i.verdict == 'TN').sum() * 100.)/len(i)
    FN = ((i.verdict == 'FN').sum() * 100.)/len(i)
    UP = ((i.verdict == 'UP').sum() * 100.)/len(i)
    UN = ((i.verdict == 'UN').sum() * 100.)/len(i)
    TQ = ((i.verdict == 'TQ').sum() * 100.)/len(i)
    FQ = ((i.verdict == 'FQ').sum() * 100.)/len(i)
    UQ = ((i.verdict == 'UQ').sum() * 100.)/len(i)
    
    resulttable.iloc[0][0] = TP
    resulttable.iloc[1][0] = FP
    resulttable.iloc[2][0] = UP
    resulttable.iloc[0][1] = TN
    resulttable.iloc[1][1] = FN
    resulttable.iloc[2][1] = UN
    resulttable.iloc[0][2] = TQ
    resulttable.iloc[1][2] = FQ
    resulttable.iloc[2][2] = UQ    
        
alltogetherpercentresulttables.append(resulttable)
display(alltogetherpercentresulttables[0])

,thread: true,thread: false,thread: query/comment
opinion: true,75,0,7
opinion: false,12,0,1
opinion: undecided,3,0,0
